In [1]:
import math
import random

def es_primo(num):
    if num < 2:
        return False
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

In [2]:
def generar_primo(bits=32):
    while True:
        candidato = random.getrandbits(bits)
        candidato |= (1 << bits - 1) | 1
        if es_primo(candidato):
            return candidato

In [3]:
def texto_a_decimal(texto):
    return int.from_bytes(texto.encode('utf-8'), 'big')

In [4]:
def decimal_a_texto(decimal):
    longitud_bytes = (decimal.bit_length() + 7) // 8
    return decimal.to_bytes(longitud_bytes, 'big').decode('utf-8')

In [5]:
def generar_claves_rsa(bits_primos=32):
    p = generar_primo(bits_primos)
    q = generar_primo(bits_primos)
    while p == q:
        q = generar_primo(bits_primos)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = 65537
    while math.gcd(e, phi_n) != 1:
        e = generar_primo(16)
    d = pow(e, -1, phi_n)
    return ((e, n), (d, n), p, q)

In [6]:
def cifrar_rsa(clave_publica, m_decimal):
    e, n = clave_publica
    if m_decimal >= n:
        raise ValueError("El mensaje es demasiado grande para este par de claves. m >= n.")
    mc = pow(m_decimal, e, n)
    return mc

In [7]:
def descifrar_rsa(clave_privada, mc):
    d, n = clave_privada
    m_decimal = pow(mc, d, n)
    return m_decimal

In [8]:
if __name__ == "__main__":
    BITS_PRIMOS = 32
    print("--- Cifrador RSA (Demostración) ---")
    print(f"ADVERTENCIA: Usando primos de {BITS_PRIMOS} bits. Esto NO es seguro para uso real.\n")
    texto_original = input("Introduce el texto a cifrar: ")
    if not texto_original:
        print("No se ha introducido texto. Saliendo.")
    else:
        try:
            m_decimal = texto_a_decimal(texto_original)
            print(f"\nTexto original: '{texto_original}'")
            print(f"Representación decimal (m): {m_decimal}")
            print("\nGenerando claves RSA...")
            clave_publica, clave_privada, p, q = generar_claves_rsa(BITS_PRIMOS)
            e, n = clave_publica
            d, _ = clave_privada
            while m_decimal >= n:
                print("El mensaje es demasiado grande para las claves generadas. Regenerando claves...")
                clave_publica, clave_privada, p, q = generar_claves_rsa(BITS_PRIMOS)
                e, n = clave_publica
                d, _ = clave_privada
            print("Claves generadas con éxito.")
            print(f"  - Módulo (n = p*q): {n}")
            print(f"  - Clave Pública (e, n): ({e}, {n})")
            print(f"  - Clave Privada (d, n): ({d}, {n})")
            print("\n--- Cifrando el mensaje ---")
            mensaje_cifrado = cifrar_rsa(clave_publica, m_decimal)
            print(f"Mensaje Cifrado (mc): {mensaje_cifrado}")
            print("\n--- Descifrando el mensaje ---")
            mensaje_descifrado_decimal = descifrar_rsa(clave_privada, mensaje_cifrado)
            print(f"Mensaje Descifrado (decimal): {mensaje_descifrado_decimal}")
            texto_final = decimal_a_texto(mensaje_descifrado_decimal)
            print("\n--- Resultado Final ---")
            print(f"Texto Original:   '{texto_original}'")
            print(f"Texto Descifrado: '{texto_final}'")
            if texto_original == texto_final:
                print("\n¡Éxito! El texto descifrado coincide con el original.")
            else:
                print("\n¡Error! El texto descifrado NO coincide con el original.")
        except ValueError as e:
            print(f"\nError: {e}")
            print("El texto introducido es demasiado largo para ser cifrado con claves de este tamaño.")
            print(f"Intenta con un texto más corto (aprox. menos de {BITS_PRIMOS*2 // 8 - 2} caracteres).")
        except Exception as e:
            print(f"Ha ocurrido un error inesperado: {e}")

--- Cifrador RSA (Demostración) ---
ADVERTENCIA: Usando primos de 32 bits. Esto NO es seguro para uso real.


Texto original: 'Holi'
Representación decimal (m): 1215261801

Generando claves RSA...
Claves generadas con éxito.
  - Módulo (n = p*q): 7736883429273487997
  - Clave Pública (e, n): (65537, 7736883429273487997)
  - Clave Privada (d, n): (561817419368311685, 7736883429273487997)

--- Cifrando el mensaje ---
Mensaje Cifrado (mc): 167361123822199060

--- Descifrando el mensaje ---
Mensaje Descifrado (decimal): 1215261801

--- Resultado Final ---
Texto Original:   'Holi'
Texto Descifrado: 'Holi'

¡Éxito! El texto descifrado coincide con el original.
